In [ ]:
%pip install -q numpy
%pip install -q pandas
%pip install -q Pillow
%pip install -q tensorflow
%pip install -q keras
%pip install -q keras-tuner
%pip install -q keras.utils
%pip install -q ipywidgets

In [163]:
import os
import numpy as np
from PIL import Image
import concurrent.futures
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [167]:
def load_images_from_folder(folder):
    images = []
    labels = []
    img_paths = []
    
    # Collect all image paths
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                if os.path.isfile(img_path) and img_path.endswith(('.png', '.jpg', '.jpeg')):
                    img_paths.append((img_path, label))
    
    # Use concurrent processing to load images
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda p: process_image(p[0]), img_paths)
    
    for (img_array, _), (_, label) in zip(results, img_paths):
        if img_array is not None:
            images.append(img_array)
            labels.append(label)
    
    return np.array(images), labels

def process_image(img_path):
    # Example function to preprocess an image (e.g., resize, normalize)
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [28, 28])
    image = image / 255.0
    return image.numpy(), img_path

# Define the paths to the folders
train_folder = 'dataset2/train'
test_folder = 'dataset2/test'
val_folder = 'dataset2/val'

# Load the images and labels
x_train, y_train = load_images_from_folder(train_folder)
x_test, y_test = load_images_from_folder(test_folder)
x_val, y_val = load_images_from_folder(val_folder)

# Encode labels if necessary
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

# Perform one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes=num_classes)

# Convert labels to numpy arrays
y_train_array = np.array(y_train_encoded)
y_test_array = np.array(y_test_encoded)
y_val_array = np.array(y_val_encoded)

ValueError: y contains previously unseen labels: '002'

In [ ]:
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers, models

# Define the hypermodel class
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = models.Sequential()
        model.add(layers.Conv2D(
            filters=hp.Int('filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('kernel_size', values=[3, 5]),
            activation='relu',
            input_shape=self.input_shape
        ))
        model.add(layers.MaxPooling2D(pool_size=2))
        model.add(layers.Flatten())
        model.add(layers.Dense(
            units=hp.Int('units', min_value=32, max_value=128, step=32),
            activation='relu'
        ))
        model.add(layers.Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

# Instantiate the hypermodel
hypermodel = MyHyperModel(input_shape=(28, 28, 3), num_classes=10)

# Instantiate the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='image_classification'
)

# Perform the hyperparameter search
tuner.search(
    x_train, 
    y_train,
    validation_data=(x_val, y_val),
    epochs=10
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Summary of the best model
best_model.summary()

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')
